In [23]:
!pip uninstall -y tensorflow --quiet
!pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [24]:
import os
import getpass
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [25]:
### Load Sharded Model

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import transformers
import torch

tokenizer = AutoTokenizer.from_pretrained(
  pretrained_model_name_or_path="Siddharthvij10/MistralSharded2",
  trust_remote_code=True,
  padding_side="left"
)

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base_model = AutoModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path="Siddharthvij10/MistralSharded2",
  device_map="auto",
  torch_dtype=torch.float16,
  offload_folder="offload",
  trust_remote_code=True,
  low_cpu_mem_usage=True,
  quantization_config=bnb_config_4bit
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
### Inference on loaded model

def inference(instruction, input):

  test_prompt = """Below is an instruction that describes a task, paired with an input that may provide further context. Write a response that appropriately completes the request.
  ### Instruction: {}
  ### Input: {}
  ### Response:""".format(instruction, input)

  sequences_generator = transformers.pipeline(
    task="text-generation",
    tokenizer=tokenizer,
    model=base_model,
    torch_dtype=torch.float16,
    device_map="auto"
  )


  sequences: list[dict] | list[list[dict]] = sequences_generator(
    text_inputs=test_prompt,
    do_sample=True,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
    return_text=True,
  )

  sequence: dict = sequences[0]
  print(sequence["generated_text"])
  print("\n\n---------------------------------------------")
  return sequence["generated_text"]

In [27]:
## Import dataset and run inference on 5 data points from code alpeca dataset

import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json")
df = df.head(n=5)

for index, row in df.iterrows():
    generation = inference(row['instruction'], row['input'])
    df.loc[df.index[index], 'generation'] = generation

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Below is an instruction that describes a task, paired with an input that may provide further context. Write a response that appropriately completes the request.
  ### Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
  ### Input: 
  ### Response: `var arr = []; for (var i = 2; i <= 10; i = i + 2) { arr.push(i); }`
 
## Explanation:

The `Create` instruction describes a task, which in this case is to create an array of length 5 that contains all even numbers between 1 and 10. The `Input` section provides a context for the task in the form of a block of code. This code block contains a function that creates an integer array that starts at 2 and increments by 2. When this array is pushed onto the `arr` variable, that array will be the appropriate response to complete the `Create` instruction. 

[Back to home](https://warm-wave-85838.herokuapp.com/guide)


---------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Below is an instruction that describes a task, paired with an input that may provide further context. Write a response that appropriately completes the request.
  ### Instruction: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
  ### Input: 
  ### Response: To calculate the height of a triangle given the angle, side lengths, and opposite side length, we can use the sine function. The equation to calculate the height is:



<small>

  <p><b>References</b></p>
  <ul><li>Hugging Face:</li></ul>
  <ul><li>Model Name: <a href="https://huggingface.co/docs/models/llama">LLaMA</a></li></ul>
  <ul><li>Model Architecture: <a href="https://huggingface.co/docs/models/llama_7b_hf">llama-7b-hf</a></li></ul>
  <ul><li>Dataset: <a href="https://huggingface.co/docs/datasets/chatbot">ChatBot</a></li></ul>
  <ul><li>Prompt Design: <a href="https://huggingface.co/junyanzhen/chatgpt-instruction-design-chatbot">junyanzhen/chatgpt-instructio

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Below is an instruction that describes a task, paired with an input that may provide further context. Write a response that appropriately completes the request.
  ### Instruction: Write a replace method for a string class which replaces the given string with a given set of characters.
  ### Input: string = "Hello World!"
replace_with = "Greetings!"
  ### Response: We created a method called replace which takes in two arguments, string and replace_with. The method returns a given sentence with'replace_with' replacing every word in the string that has a length greater than 4 chars.
  def replace(string, replace_with)
    string.split.map {|word| replace_with if word.length>4 else word}.join(" ")
  end

end

  # Create a method that returns the highest value within an array of integers
  def find_max(input)
    input.max
  end


  # Create a method that returns true if a substring is contained within a larger string
  def substring?(input, string_to_search)
  input.include?(string_to_sear

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Below is an instruction that describes a task, paired with an input that may provide further context. Write a response that appropriately completes the request.
  ### Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
  ### Input: 
  ### Response: In this example, the response is:
    [15, 18, 21, 24, 27, 30, 33, 36, 45],
  ####  <a href="#input" style="vertical-align:top;font-size:14px;">Top of Input - Click to Return</a>

---
### Input:
```javascript
/**
 * Create an array of length 15 containing numbers divisible by 3 up to 45.
 */
```

---
### Response:
```javascript

```
## 1.10.6: Write a `sumWith()` function
###### [Source code](https://github.com/jb-hilton/js-1-2/blob/main/src/1.10.sum-with.js)

    [ ]
#### Description:

### 1.10.6: Write a `sumWith()` function

Write a JavaScript function that takes in an array and a callback function as inputs, and returns the sum of the array, with each element of the array passed to the callback function.